In [17]:
%load_ext autoreload
%autoreload 2
import sys
dirr = "../../src"
if dirr not in sys.path:
    sys.path.append(dirr)
import pandas as pd
import numpy as np
import numpy.random as npr
import utils as helper
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Q2
Custom functions for differnt ways to calculate theta.

In [74]:
# reading data
data_loc = "../../data/real_estate/dataset.xlsx"
data = pd.read_excel(data_loc)
X = data.iloc[:, 1:-1]
y = data.iloc[:, -1]

# Normalizing the data
scaler = MinMaxScaler()
X[X.columns] = scaler.fit_transform(X[X.columns])
tupl = train_test_split(X.values, y.values, test_size = .3)
X_train, X_test, y_train, y_test = tupl
X_train[:3, :]

/home/apoorv/miniconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


array([[0.72727271, 0.18949772, 0.0125958 , 0.5       , 0.42014057,
        0.72395946],
       [0.27272729, 0.29680365, 0.07252509, 0.5       , 0.40087252,
        0.68837611],
       [0.72727271, 0.58447489, 0.69552341, 0.        , 0.19619486,
        0.24088851]])

In [77]:
from lregress import normalEquationRidgeRegression
theta = normalEquationRidgeRegression(X_train, y_train, lmbd=0.1)
print ("normalEquationRidgeRegression:\n", theta)

normalEquationRidgeRegression:
 [ 30.8772563    4.02902254 -10.00324397 -28.90024718  11.62381309
  18.16719681   1.08461451]


In [80]:
# from lregress import normalEquationRidgeRegression
# print(type(y_train))
# for i in range(len(y_train)):
#     print (y_train[i])
theta = sgdRegression(X_train, y_train, alpha = 0.01, it=100)
print ("sgdRegression:\n", theta)


sgdRegression:
 [10.73818571 10.53775786 11.3733753  11.39333645 11.06786246 10.83888066
 10.98884252]


In [61]:
from utils import prepend_ones_col
def sgdRegression(X, y, alpha = 0.001, it=100):
    X_new = prepend_ones_col(X)
    n, m = X_new.shape
    th = np.random.rand(m)
    for i in range(it):
        for j in range(n):
            yhat = (X_new[j, :] * th).sum()
            e = y[j]-yhat
            weighted_e = X_new[j, :] * e
            sumz = weighted_e.sum()
            th = th + (2*alpha*(sumz))
    return th

---

# Q5
Using sklearn's RIDGE and LASSO modules after normalizing the data.

**a)** 

In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

The RMSE error obtained is:  10.809957718336143


In [ ]:

def cross(dTrain, depths, num_valid):
    folds = make_folds(dTrain, num_valid)
    passed_folds = folds[:-1] # holding out test
    k = best_k(passed_folds, depths)
    acc = 0
    curr_fold_train = folds[:-1]
    curr_fold_test = folds[-1]
    linear_reg = Ridge(alpha=k)
    linear_reg.fit(curr_fold_train.iloc[:, :-1], curr_fold_train.iloc[:, -1])
    my_preds = np.squeeze(mydt.predict(curr_fold_test.iloc[:, :-1]).values)
    true = curr_fold_test.iloc[:, -1].values
    my_acc = ((true == my_preds).sum())/curr_fold_test.shape[0]
    acc += my_acc/num_valid
    return {"depth": k, "acc": acc}   

def make_folds(dTrain, num_valid):
    '''Make a number of folds with diven pd'''
    train = shuffle(dTrain)
    last = train.shape[0]
    folds = []
    for i in range(num_valid):
        start = i*int(last/num_valid)
        end = (i+1)*int(last/num_valid)
        if end >= last:
            end = -1
        folds.append(train.iloc[start:end, :])
    return folds

def best_k(folds, depths):
    '''return depth that maximizes the avg accuracy'''
    num_valid = len(folds)
    acc = {}
    for i in range(num_valid):
        curr_fold_train = [x for j,x in enumerate(folds) if j!=i] 
        curr_fold_validation = folds[i]
        curr_fold_train = pd.concat(curr_fold_train)
        for k in depths:
            linear_reg = Ridge(alpha=k)
            linear_reg.fit(curr_fold_train.iloc[:, :-1], curr_fold_train.iloc[:, -1])
            my_preds = np.squeeze(mydt.predict(curr_fold_validation.iloc[:, :-1]).values)
            true = curr_fold_validation.iloc[:, -1].values
            my_acc = ((true == my_preds).sum())/curr_fold_validation.shape[0]
            if k in acc.keys():
                acc[k] += my_acc
            else:
                acc[k] = my_acc
    acc = {k: acc[k]/num_valid for k in acc.keys()}
    
    x = acc
    print (acc) # for showcasing
    sorted_by_value = sorted(x.items(), key=lambda kv: kv[1])
    return sorted(x[0] for x in sorted_by_value if sorted_by_value[-1][1] == x[1])[0]





lr = Ridge(alpha=1.0)
lr.fit(X_)

def cross_val(master_data, clf, fol=5):
    dFull = master_data.copy()
    kf = KFold(n_splits=fol, shuffle=True)

    train_rmse = {}
    val_rmse = {}
    for i, (ixTrain, ixVal) in enumerate(kf.split(dFull)):
        # combining the folds
        fTrain, fVal = dFull.iloc[ixTrain], dFull.iloc[ixVal]
        
        # training on train and getting predictions on val, and train
        clf.fit(fTrain.iloc[:, :-1], fTrain.iloc[:, -1])
        y_pred_val = clf.predict(fVal.iloc[:, :-1])
        y_pred_train = clf.predict(fTrain.iloc[:, :-1])
        
        
        # getting the rmse values for train and val
        r_val = helper.rmse(y_pred_val, y_test_set.values)
        r_train = helper.rmse(y_pred_train, fTrain.iloc[:, -1].values)
        
        
        train_rmse[ixTrain] = r_val 
        val_rmse[ixVal] = r_train
        
    bst_fea_till_now = list(min(saved_rmse, key=saved_rmse.get))
    curr_rmse = saved_rmse[tuple(bst_fea_till_now)]
    if (best_rmse > curr_rmse + tol):
        saved_fea = bst_fea_till_now
        best_rmse = curr_rmse
    else:
        break

    # getting the best set of features, (min rms)
    best_feature = dTrain.columns[list(min(saved_rmse, key=saved_rmse.get))]
    print("Best Feature Set: {}".format(list(best_feature)))

    reg_org = LinearRegression().fit(dTrain.iloc[:, best_feature], dTrain.iloc[:, -1])
    y_pred = reg_org.predict(dTest.iloc[:, best_feature])
    err = y_pred - dTest.iloc[:, -1].values
    rms_err = np.sqrt(np.square(err).sum()/len(y_pred))
    print(rms_err)

    print("Complete Feature Set: {}".format(list(dTrain.columns)))
    reg_op = LinearRegression().fit(dTrain.iloc[:, :-1], dTrain.iloc[:, -1])
    y_pred = reg_op.predict(dTest.iloc[:, :-1])
    err = y_pred - dTest.iloc[:, -1].values
    rms_err = np.sqrt(np.square(err).sum()/len(y_pred))
    print(rms_err)

y_pred = lr.predict(X_test)
print("The RMSE error obtained is: ", helper.rmse(y_pred, y_test.values))


In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10,6.15))
map_X, map_Y = np.meshgrid(theta0_grid, theta1_grid)
ax[0].scatter(x, Y, marker='x', s=40, color='k')
contours = ax[1].contour(map_Y, map_X, J_grid, 30)
ax[1].clabel(contours)
ax[1].scatter([theta0_true]*2,[theta1_true]*2,s=[50,10], color=['k','w'])
ax[1].set_xlabel(r'$\theta_0$')
ax[1].set_ylabel(r'$\theta_1$')
ax[1].set_title('Cost function')
ax[0].set_xlabel(r'$x$')
ax[0].set_ylabel(r'$y$')
ax[0].set_title('Data and fit')
line, = ax[0].plot([], [], color='r', lw=2,label=r'$\theta_0 = {:.3f}, \theta_1 = {:.3f}$'.format(*Theta[0]))
def animate(i):
   '''ax[0].plot(X, predict(Theta[i], X), color='r', lw=2,
              label=r'$\theta_0 = {:.3f}, \theta_1 = {:.3f}$'.format(*Theta[i]))'''
   #ax[1].scatter(Theta[i][0], Theta[i][1], s=40, lw=0)
   z = predict(Theta[i], X)
   line.set_data(X[:, 1], z)
   ax[1].annotate('', xy=Theta[i], xytext=Theta[i-1],arrowprops={'arrowstyle': '->', 'color': 'r', 'lw': 1}, va='center', ha='center')
   #ax[0].plot(X, predict(Theta[i], X), color='r', lw=2,label=r'$\theta_0 = {:.3f}, \theta_1 = {:.3f}$'.format(*Theta[i]))
   return ax

#anim = animation.FuncAnimation(fig, animate, frames=10, interval=20, blit=True)
anim = animation.FuncAnimation(fig, animate, frames=1000, interval=20, blit=True)
#HTML(anim.to_html5_video())
anim